In [1]:
import jieba
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import ttk

In [2]:
def text_to_vector(text, model):
    vector_list = [model.wv[word] for word in text if word in model.wv]
    if len(vector_list) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vector_list, axis=0)

def split_text(user_input):
    text_generater = jieba.cut(user_input,cut_all=False)
    result = ' '.join(text_generater)
    return result

def convert_str_to_array(s):
    return np.fromstring(s[1:-1], sep=' ')
word2vec_model = Word2Vec.load(r"C:\Users\fento\Downloads\data_cleaned\word2vec.model")

In [3]:
def recommend_games(user_input, games_data, top_n=10):
    print(f"Inside recommend_games with user_input: {user_input}")
    processed_input = split_text(user_input)
    input_vector = text_to_vector(processed_input, word2vec_model)
    if np.all(input_vector == 0):
        return "Could not process user input."
    
    input_vector = input_vector.reshape(1, -1)
    tags_similarity = cosine_similarity(input_vector, np.stack(games_data['tags_vector'].apply(convert_str_to_array)))
    description_similarity = cosine_similarity(input_vector, np.stack(games_data['description_vector'].apply(convert_str_to_array)))
    reviews_similarity = cosine_similarity(input_vector, np.stack(games_data['reviews_vector'].apply(convert_str_to_array)))
    sentiment_score = games_data['Sentiment_Score']
    sentiment_score_array = sentiment_score.values.reshape(-1, 1)
    tags_weight = 0.3
    description_weight = 0.3
    reviews_weight = 0.3
    sentiment_weight = 0.1

    avg_similarity = (tags_similarity * tags_weight + description_similarity * description_weight + 
                  reviews_similarity * reviews_weight + sentiment_score_array * sentiment_weight) / (tags_weight + description_weight + reviews_weight + sentiment_weight)
    
    top_n_indices = avg_similarity[0].argsort()[-top_n:][::-1]
    top_n_games = games_data.iloc[top_n_indices]
    if top_n_games.empty:
        return "No games found."
    
    return top_n_games[['名字', 'tags' , '描述', '價格']]

path = 'C:\\Users\\fento\\Downloads\\data_cleaned\\gamesdata_280_2.xlsx'
stop_words_path = 'C:\\Users\\fento\\Downloads\\gameID_reviews_cleaned\\stop_words_3.txt'
games_data = pd.read_excel(path,index_col=0)

In [4]:
# 處理按鈕點擊的函數
def on_click():
    user_input = user_entry.get()
    recommended = recommend_games(user_input, games_data)
    result_text.delete('1.0', tk.END)  # 清空先前的推薦結果
    if isinstance(recommended, str):
        # 如果是字符串（表示出錯或找不到遊戲），直接顯示
        result_text.insert(tk.END, recommended)
    else:
        # 將 DataFrame 轉換為字符串以顯示在 Text 控件上
        text_to_display = ""
        for _, row in recommended.iterrows():
            text_to_display += f"名字：{row['名字']}\ntags：{row['tags']}\n描述：{row['描述']}\n價格：{row['價格']}\n---\n\n"
        result_text.insert(tk.END, text_to_display)

# 初始化 Tkinter
root = tk.Tk()
root.title("遊戲推薦系統")

# 添加標籤和文本框
prompt_label = ttk.Label(root, text="請輸入您喜歡的遊戲類型或特點：")
prompt_label.pack(pady=10)
user_entry = ttk.Entry(root, width=80)
user_entry.pack(pady=10)

# 添加按鈕
recommend_button = ttk.Button(root, text="獲得推薦", command=on_click)
recommend_button.pack(pady=10)

# 添加用於顯示推薦結果的標籤
result_label = ttk.Label(root, text="")
result_label.pack(pady=10)

result_text = tk.Text(root, wrap=tk.WORD, width=80, height=30)
result_text.pack(side=tk.LEFT, fill=tk.Y)
scrollbar = ttk.Scrollbar(root, orient=tk.VERTICAL, command=result_text.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
result_text.config(yscrollcommand=scrollbar.set)

# 運行 GUI
root.mainloop()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\fento\AppData\Local\Temp\jieba.cache


Inside recommend_games with user_input: 瘋狂殺殭屍


Loading model cost 0.436 seconds.
Prefix dict has been built successfully.


Inside recommend_games with user_input: 找一款可以瘋狂殺殭屍 殺到爽的遊戲
Inside recommend_games with user_input: 找一款可以瘋狂打殭屍的遊戲
Inside recommend_games with user_input: 找一款可以瘋狂射殭屍的遊戲
Inside recommend_games with user_input: 找一款瘋狂射殭屍的遊戲
Inside recommend_games with user_input: 瘋狂射殭屍的遊戲
Inside recommend_games with user_input: 瘋狂射殭屍
Inside recommend_games with user_input: 瘋狂射殺殭屍
